## CS5304/INFO5304 - Data Science: Airbnb Project
William Tung - wt275
### EDA & Data Cleaning

In [1]:
# Packages
import numpy as np
import pandas as pd

#### Preliminary EDA & Dataset Overview

In [2]:
# Airbnb NYC 2019 Data
data = pd.read_csv('data/AB_NYC_2019_CLEAN.csv')
print('Dataset Shape:',data.shape)
data.describe()

Dataset Shape: (48895, 16)


,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.090910,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.597283,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.040000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.370000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,1.580000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


##### Columns Types

In [3]:
print('Columns and dtype')
data.dtypes

Columns and dtype


id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

##### Missing Values Count
We can see from the missing value counts per column that the dataset appears to be relatively clean except for four columns. 'last_review' and 'reviews_per_month' both have an equally high 10,052 missing values, while 'name' and 'host_name' are missing a respective 16 and 21 entries. All other columns do not have any missing values as marked with null or N/A. Upon checking the other columns and their descriptive purpose, we can conclude that the missing value notation for this dataset is 'NaN' and all other zero value entries appear to be valid for the respective columns.

As for missing value handling, we can leave 'name' and 'host_name' as is since those fields aren't crucial to identify the listing or the host which both have a corresponding ID field. Potential analysis related to these name fields could revolve on the 'attractiveness' of the listing name or possible bias towards a host name. Segmentation and consideration will be given to these fields if that type of analysis is explored. 

Since the columns 'last_review' and 'reviews_per_month' are quantitative fields, we will need to fill values for analysis. It is likely that these missing value entries are related to listing that have not had a review or a guest yet. Both of these described scenarios is completely valid. 

In [4]:
print('Columns and Missing Value Counts')
data.isna().sum()

Columns and Missing Value Counts


id                                 0
name                              16
host_id                            0
host_name                         21
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                     0
number_of_reviews                  0
last_review                        0
reviews_per_month                  0
calculated_host_listings_count     0
availability_365                   0
dtype: int64

In [5]:
# Testing no reviews for missing NaN review columns
print('# of common occurrences:',len(data[(data['number_of_reviews'] == 0)\
    & (data['last_review'].isna()) & (data['reviews_per_month']).isna()]))

# of common occurrences: 0


##### Missing Value Handling
Infact there are no reviews (0) for when 'number_of_reviews' and 'reviews_per_month' is NaN, as both the NaN count and common occurrence count is 10,052. Our method to fill these NaN entries will be to replace NaN with zeros, so we can use these columns for quantitative analysis. Choosing zero as the replacement value is valid for the meaning of these columns and also follows the 'number_of_reviews' notation of having zero values as well.

In [6]:
# Creating a clean copy of the dataset
data_clean = data.copy()
# Fill Nan Values with 0
data_clean['last_review'] = data_clean[['last_review']].fillna(0) # last_review
data_clean['reviews_per_month'] = data_clean[['reviews_per_month']].fillna(0) # last_review
# Check Missing Values
data_clean.isna().sum()

id                                 0
name                              16
host_id                            0
host_name                         21
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                     0
number_of_reviews                  0
last_review                        0
reviews_per_month                  0
calculated_host_listings_count     0
availability_365                   0
dtype: int64

In [7]:
# data_clean summary
data_clean.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,0,0.00,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


##### Write data_clean csv

In [8]:
# Output file
data_clean.to_csv('data/AB_NYC_2019_CLEAN.csv',index=False)
print('data_clean output to data/AB_NYC_2019_CLEAN.csv')

data_clean output to data/AB_NYC_2019_CLEAN.csv


In [9]:
dhoods = data['neighbourhood'].unique()

In [10]:
listings = pd.read_csv('data/listings.csv')
listings.head()

/Users/williamtung/anaconda3/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2060,https://www.airbnb.com/rooms/2060,20200408162728,2020-04-09,Modern NYC,NaN,"Lovely, spacious, sunny 1 BR apartment in 6th ...","Lovely, spacious, sunny 1 BR apartment in 6th ...",none,NaN,...,f,f,flexible,f,f,1,0,1,0,0.01
1,2595,https://www.airbnb.com/rooms/2595,20200408162728,2020-04-10,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,...,f,f,strict_14_with_grace_period,t,t,2,2,0,0,0.38
2,3831,https://www.airbnb.com/rooms/3831,20200408162728,2020-04-11,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,...,f,f,moderate,f,f,1,1,0,0,4.68
3,5099,https://www.airbnb.com/rooms/5099,20200408162728,2020-04-10,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,I have a large 1 bedroom apartment centrally l...,My large 1 bedroom apartment has a true New Yo...,none,My neighborhood in Midtown East is called Murr...,...,f,f,moderate,t,t,1,1,0,0,0.58
4,5121,https://www.airbnb.com/rooms/5121,20200408162728,2020-04-09,BlissArtsSpace!,NaN,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,1,0,1,0,0.38


In [11]:
listings['zipcode'].unique()

array(['10040', '10018', '11238', '10016', '11216', '10019', '10025',
       '10002', '10036', '10014', '11215', '11211', '10029', '11205',
       '11217', '10034', '10003', '10031', '10027', '11237', '11226',
       '10009', '11225', '11101', '11221', '11206', '11222', '10010',
       '11249', '10001', '10162', '11231', '10035', '10032', '11377',
       '11201', '11233', '10023', '10128', '11234', '11367', '10024',
       '10028', '10069', '11104', '10065', '10301', '10013', '10452',
       '10039', '11385', '10021', '10011', '10075', '11434', '11379',
       '10026', '11105', '10044', '10022', nan, '11103', '11218', '10473',
       '10475', '10468', '11102', '11213', '10033', '11427', '11693',
       '10012', '11375', '10470', '10030', '11207', '10453', '11223',
       '10469', '11208', '11235', '10314', '11209', '11204', '10305',
       '11691', '11368', '11232', '10017', '11373', '11224', '11412',
       '10004', '11203', '11374', '10466', '10038', '11106', '10304',
       '11220',

In [12]:
"""neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude"""
lhoods = listings['neighbourhood_cleansed'].unique()

count = 0
for d in dhoods:
    if d in lhoods:
        count+=1

print('%s/%s Matches Found'%(count,len(dhoods)))
print(len(lhoods))

221/221 Matches Found
223


In [33]:
ref = listings[['neighbourhood_cleansed','zipcode']]

In [34]:
ref

,neighbourhood_cleansed,zipcode
0,Washington Heights,10040
1,Midtown,10018
2,Clinton Hill,11238
3,Murray Hill,10016
4,Bedford-Stuyvesant,11216
...,...,...
50373,Chelsea,10001
50374,Sunnyside,11377
50375,Astoria,11102
50376,Hell's Kitchen,10036


In [46]:
reference = ref.groupby(['neighbourhood_cleansed','zipcode'])

In [54]:
out = reference.groups.keys()
out

dict_keys([('Washington Heights', '10040'), ('Midtown', '10018'), ('Clinton Hill', '11238'), ('Murray Hill', '10016'), ('Bedford-Stuyvesant', '11216'), ("Hell's Kitchen", '10019'), ('Upper West Side', '10025'), ('Chinatown', '10002'), ("Hell's Kitchen", '10036'), ('West Village', '10014'), ('South Slope', '11215'), ('Williamsburg', '11211'), ('East Harlem', '10029'), ('Fort Greene', '11205'), ('Crown Heights', '11216'), ('Park Slope', '11217'), ('Inwood', '10034'), ('East Village', '10003'), ('Harlem', '10031'), ('Harlem', '10027'), ('Bedford-Stuyvesant', '11238'), ('Bushwick', '11237'), ('Flatbush', '11226'), ('Lower East Side', '10002'), ('East Village', '10009'), ('Prospect-Lefferts Gardens', '11225'), ('Long Island City', '11101'), ('Bedford-Stuyvesant', '11221'), ('Williamsburg', '11206'), ('Greenpoint', '11222'), ('Kips Bay', '10010'), ('Kips Bay', '10016'), ("Hell's Kitchen", '10018'), ('Nolita', '10002'), ('Williamsburg', '11249'), ('Chelsea', '10001'), ('Upper East Side', '101

In [52]:
listings['zipcode'].isna().sum()

440

In [55]:
out.to_csv('data/zip_ref.csv',index=False)

AttributeError: 'dict_keys' object has no attribute 'to_csv'

In [1]:
import geopy
import pandas as pd


def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']['postcode']

In [2]:

geolocator = geopy.Nominatim(user_agent='my-application')

df = pd.DataFrame({
    'Lat': [29.39291, 29.39923, 29.40147, 29.38752, 29.39291, 29.39537, 29.39343, 29.39291, 29.39556],
    'Lon': [-98.50925, -98.51256, -98.51123, -98.52372, -98.50925, -98.50402, -98.49707, -98.50925, -98.53148]
})
zipcodes = df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='Lat', lon_field='Lon')

In [3]:
zipcodes

0    78204
1    78204
2    78204
3    78225
4    78204
5    78204
6    78204
7    78204
8    78225
dtype: object